In [1]:
import pandas as pd
import numpy as np
import pyreadstat

In [2]:
path_spss='C:/Users/Jcabr/Desktop/Notebooks/spss file/Crt - Mx - Cum file - Jun21 - 16Jul21 JC.sav'
####types_var_path='C:/Users/Jcabr/Desktop/Notebooks/Checks folder'
check_folder='C:/Users/Jcabr/Desktop/Notebooks/Checks folder'
output_path='C:/Users/Jcabr/Desktop/Notebooks/Output folder'
output_check_spss='C:/Users/Jcabr/Desktop/Notebooks/input_folder/TABLES MX Excel.xlsx'

In [3]:
#LOADING FILE 
#TASK - PROBLEM WITH MEMORY


#df=pd.read_spss('C:/Users/JavierCabrera/Desktop/NT/78 - jun -21/CRT - Global - Jul20-Jun21 - cumulative working file CLEANED.sav')
#df_val,meta=pyreadstat.read_sav('C:/Users/JavierCabrera/Desktop/NT/78 - jun -21/CRT - Global - Jul20-Jun21 - cumulative working file CLEANED.sav', apply_value_formats=False)


df_lab=pd.read_spss(path_spss)
df,meta=pyreadstat.read_sav(path_spss, apply_value_formats=False)

df_var_labels=df_lab.columns=meta.column_labels
df_lab=pd.read_spss(path_spss)



#df_orig=df

var_list=list(df.columns.values)
#df_var_labels



In [4]:
###Global functions

### Dictionary function

def crear_diccionario(col1, col2):
    a=dict(zip(col1,col2))
    return a

#Function to put one column in first position

def order_function(x,y):
    l1=x.copy()
    i1=x.index(y)
    el1=[l1[i1]]
    l1.remove(el1[0])
    return el1 + l1

### bold function

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [5]:
# Creating dict variable labels 

if len(var_list)==len(df_var_labels):
    dict_var=crear_diccionario(var_list, df_var_labels)
    
df_dict_var=pd.DataFrame.from_dict(dict_var,orient='index')   


# extract dict for variables and respective labels
#df_dict_var.to_excel('C:/Users/JavierCabrera/Desktop/Notebooks/Pivot tables project/Output folder/dictionary variables.xlsx',sheet_name='dict_var')

In [6]:
# control number variables and check structure

shape_df=df.shape
shape_df_lab=df.shape

if (shape_df==shape_df_lab):
    ncolumns=shape_df[1]
else:
    print("Different number of columns or rows")
print ("Same number of variables in both files = " + str(ncolumns) + " variables")

Same number of variables in both files = 969 variables


In [7]:
# CONTROL VARIABLE TYPES 
# Inst -  See the printed list and check you don't have futher than category, float, object and int

types_check=df_lab.dtypes
#df_lab.info()
types_check.to_excel(check_folder + 'Type variables check.xlsx',sheet_name='var_type')
types_check_int=pd.read_excel(check_folder + 'Type variables check.xlsx',sheet_name='var_type', engine='openpyxl')
check_cat=list(types_check_int.iloc[:,1].unique())
check_cat


['category', 'float64', 'object']

In [8]:
##DICCIONARIO  values creation

Val_lab=dict()

for i in range(ncolumns):
    variable=df.columns[i]
    var_lab=df_lab.columns[i]
            
    if (i==999999999999):
        break
    
    if variable!=var_lab:
        print('var not found or not in same order')
        break
    if df_lab[var_lab].dtype=='int':
        continue    
    if df_lab[var_lab].dtype=='float':
        continue
    if df_lab[var_lab].dtype=='object':
        continue
    if df_lab[var_lab].dtype=='category':
        df_dict = df[[variable]].merge(df_lab[[variable]], left_index=True, right_index=True)
        df_dict=df_dict.drop_duplicates()
        Val_lab[variable]={c:v for c,v in zip(df_dict.iloc[:,0], df_dict.iloc[:,1])}
        #print(var_lab,'should be renamed')

##DICCIONARIO  WAVES CREATION
        
        
waves_dict=dict()
waves_labs = df[['Month']].merge(df_lab[['Month']], left_index=True, right_index=True)
waves_labs =waves_labs.drop_duplicates()
waves_dict={c:v for c,v in zip(waves_labs.iloc[:,0], waves_labs.iloc[:,1])}


#print(variable)
#print(var_lab)
#print(Val_lab)



In [10]:
def dict_apply_Weights(test):
    
    ###function to apply dicts - function include when calculating scores

    
    table_var=list(test.columns.values)
    counter=0
    
    for i in table_var:
    
        counter=counter+1
    
        if i=="Variable":
            continue
        try:
            test=test.replace({i:Val_lab[i]})
        
        except:
            print("")
            #test=test.rename(columns={i:list(waves_dict.values())[counter-(num_var + 2)]})

    test=test.replace({"Variable":dict_var})

    return test 

In [11]:
#1- prod - function to calculate scores and validn weighted

import warnings
warnings.filterwarnings("ignore")#, message="divide by zero encountered in true_divide")

def weight():
    
    
    
    #df=[]
    #df=df_orig
    
    
    
    counter=0
    
    lista0=head.copy()
    lista0.extend(period)

    
    
    for i in var:
        
        #######
        df["Weight_Final"]=df["Weight_Final2"]
        df["Weighted_cells"]=df[i]*df["Weight_Final"]
        df.loc[np.isnan(df[i]),"Weight_Final"]=0 
        
        weigth_temp=df.groupby(lista0)["Weighted_cells"].sum().values
        suma_temp=df.groupby(lista0)[["Weight_Final"]].sum().values
        #suma_temp=df[df["Value"].notnull()].groupby(head)[["Weight_Final"]].sum().values

        
        table=df.groupby(lista0)[i].mean()
        table=table.reset_index()

        try:

            table["Mean"]=np.diag(weigth_temp/suma_temp)
        except ZeroDivisionError:
            table["Mean"]=None
            ignore

        ###weigted counts
        table["ValidN"]=suma_temp
    
        ###Remove unweigthed score and substitute by var_name
        table[i]=i
        
        ###rename var name
        table.rename(columns={i:"Variable"},inplace=True)
        
        if counter==0:
            table_final=table
        else:

            table_final=pd.concat([table_final,table],ignore_index=False)
        counter=counter+1

    

    #apply dict funcion
    table_final=dict_apply_Weights(table_final)
    
    #decimals control (exlcuded no need this remove and check)
    table_final['Mean']=table_final['Mean']
    table_final['ValidN']=table_final['ValidN']
     
    #table_final['ValidN']=table_final['ValidN'].astype(int)
    
    df["Weight_Final"]=df["Weight_Final2"]
    
    return pd.DataFrame(table_final)

In [12]:
##FUNCTION TO PUT PERIOD IN COLUMNS

def period_col(result_final,period):
    #FUNCTION TO PUT PERIODS IN COLUMNS (FROM ROWS)
    
    global periods_order #we use period order variable in check program
    global lista2

    
    
    
    period_str="".join(period)
    periods_order=list(result_final["Month"].unique())
    periods=list(reversed(periods_order))

    lista2=head.copy()
    lista2.extend(["Variable"])

    counter=0



    for i in periods_order:
        result_final_p=result_final[result_final[period_str]==i]
        result_final_p=result_final_p.rename(columns={"Mean":"Mean_" + str(i)})
        result_final_p=result_final_p.rename(columns={"ValidN":"Valid N_" + str(i)})
        result_final_p.drop(period,axis='columns',inplace=True)
        if counter==0:
            #break
            result=result_final_p
        else:
            result=result.merge(result_final_p,how='outer',on=lista2)


        counter=counter+1

    return result   
    #result.to_excel('C:/Users/JavierCabrera/Desktop/DS/Notebooks/Pivot tables project/Output_Scores weighted.xlsx',sheet_name='Scores weighted')

In [13]:
def Periods_MoM(test):
    #Function to calculate periods 3M 6M 9M 12M when having 12 months of data
    
    test['ValidN_L3M']=test.filter(regex=r'^Valid N.*[0-9]$').iloc[:,-3:].sum(skipna=True, axis=1)
    test['Mean_L3M']=test.filter(regex=r'^Mean.*[0-9]$').iloc[:,-3:].mean(skipna=True, axis=1)

    test['ValidN_L6M']=test.filter(regex=r'^Valid N.*[0-9]$').iloc[:,-6:].sum(skipna=True, axis=1)
    test['Mean_L6M']=test.filter(regex=r'^Mean.*[0-9]$').iloc[:,-6:].mean(skipna=True, axis=1)

    test['ValidN_L9M']=test.filter(regex=r'^Valid N.*[0-9]$').iloc[:,-9:].sum(skipna=True, axis=1)
    test['Mean_L9M']=test.filter(regex=r'^Mean.*[0-9]$').iloc[:,-9:].mean(skipna=True, axis=1)

    test['ValidN_L12M']=test.filter(regex=r'^Valid N.*[0-9]$').iloc[:,-12:].sum(skipna=True, axis=1)
    test['Mean_L12M']=test.filter(regex=r'^Mean.*[0-9]$').iloc[:,-12:].mean(skipna=True, axis=1)
    
    return test

In [14]:
def table_function():
        
        #this function calls all previous functions and aplly it for each cut selected
            #weight - for weighting and scores wiht validn
            #period_col - for putting period in columns
            #periods_mom - for calculating MOM - 3M TO 12 M - 
        
        
    global head
    global table_final3 #remove this - not used
    
    counter_cut=0
    
    for x in cuts:
        
        head=setup + [x]
    
        table=weight()
        table2=period_col(table,period)
        table_final2=Periods_MoM(table2)
        
        table_final2.rename(columns={x:"Cut Level"},inplace=True)
        
        #####TASK APPLY DICT
        
        table_final2["Cut"]=x
        table_final2=table_final2.replace({"Cut":dict_var})
        
        
    #do the merge of the several tables based on cuts
    
        if counter_cut==0:
        
            table_final3=table_final2
        
        else:
           
            table_final3=pd.concat([table_final3,table_final2],ignore_index=False)
        
        counter_cut=counter_cut+1
    
    # order new Cut variable name after setup.(relationship with 'function order' function)
    
    cols=list(table_final3.columns)
    table_final3=table_final3[order_function(cols,"Cut")]

    for i in reversed(setup):
        
        cols=list(table_final3.columns)
        table_final3=table_final3[order_function(cols,i)]

    
    
    ###possible problem - if in table name we have several tables as a list - globals do not work with list and we use join. also we need to allow names with spaces
    
    table_name_string="".join(table_name)
    globals()[table_name_string] = table_final3
    
    ###PROGRAM FOR CHECK
    
    if Check_py_xls=="Y":
        excel_table_formatting()
        check_func2_order()
        check_func3_checks()
        
    #return pd.DataFrame(table_final3) ###no need this return


In [15]:
def excel_table_formatting():
    ###LOADING XLS AND FORMATING -- we load excel file, remove empty rows, name columns as we do in python and control dtypes
    
    
    ###check when overall only 


    include_cut=0
    
    global table_xls_check
    
    #load excel from spss sheet list 1
    
      
    table_xls_check=pd.read_excel(output_check_spss,sheet_name=tables_spss[0], engine='openpyxl')
    
    
    #FORMATING EXCEL TABLE
        
    
    # 1 removing empty initial rows    
    check_inirow=0
    total_columns=len(table_xls_check.iloc[0])
    
    
    # Removing empty rows
    while check_inirow<3:  ###when if found more than 2 not empty raws in excel it defines at first row of the spss. 
            
        row1=table_xls_check.iloc[0]
        check_inirow=row1.isnull().sum()
        check_inirow=total_columns-check_inirow
            
        if check_inirow<2: # If only 1 not missing line is removed
                
            table_xls_check.drop(table_xls_check.iloc[0].name, inplace=True)

            
            
    #####putting names to the columns in excel

    #task - add exception when at right of variable we have level (apply to percentages)

    summer=0

    for z in setup:


        table_xls_check.rename(columns={ table_xls_check.columns[summer]: setup[summer] },inplace=True)

        summer=summer+1

    #cut column
    ######len(Scores["Cut"].unique()) we would have to use this one in order to use the cut of the table we will use



    if len(cuts)>1:    
        table_xls_check.rename(columns={ table_xls_check.columns[summer]: "Cut" },inplace=True)   
        summer=summer+1

    if len(cuts)==1:

        check_cut_col=table_xls_check.iloc[:,summer]
        check_cut_col=len(check_cut_col.dropna().unique())

        if(check_cut_col)==1: ###checking if exist the cut variable

            table_xls_check.rename(columns={ table_xls_check.columns[summer]: "Cut" },inplace=True)   
            summer=summer+1
        else:
            include_cut=1

    #cut_level

    table_xls_check.rename(columns={ table_xls_check.columns[summer]: "Cut Level" },inplace=True)
    summer=summer+1


    ##checking battery column existence

    check_battery_col=table_xls_check.iloc[:,summer]
    check_battery_col=len(check_battery_col.dropna().unique())

    if check_battery_col==1:

        #task meter un try except o un bucle para ver si existe battery.
        table_xls_check.rename(columns={ table_xls_check.columns[summer]: "Battery" },inplace=True)
        summer=summer+1

    #variable column

    table_xls_check.rename(columns={ table_xls_check.columns[summer]: "Variable" },inplace=True)
    summer=summer+1

    ####headers on periods

    for w in range(len(table_xls_check.columns) - summer):
        try:
            table_xls_check.rename(columns={ table_xls_check.columns[summer]: table_xls_check.iloc[1][summer] + "_" + table_xls_check.iloc[0][summer] },inplace=True)  
            summer=summer+1
            table_xls_check.rename(columns={ table_xls_check.columns[summer]: table_xls_check.iloc[1][summer] + "_" + table_xls_check.iloc[0][summer-1] },inplace=True)
            summer=summer+1

        except Exception:
            pass



    ###create cut var if not created

    if include_cut==1:
        table_xls_check["Cut"]=cuts[0] 

        cols=list(table_xls_check.columns)
        table_xls_check=table_xls_check[order_function(cols,"Cut")]

        for i in reversed(setup):
            cols=list(table_xls_check.columns)
            table_xls_check=table_xls_check[order_function(cols,i)]

    ###removing columns with all empty

    table_xls_check=table_xls_check.dropna(1,'all')

    #removing 2 first rows from month and indicator
    table_xls_check=table_xls_check.drop([1,2], axis=0)
    

    ###changing type of vars in excel
    

    for i in table_xls_check.columns:
        try:
            table_xls_check[i] = table_xls_check[i].astype('float64')
        except ValueError:
            continue
    


In [16]:
####ORDER TABLES ON PY + XLS

def check_func2_order():
    global table_xls_check
    global table_py_check

    table_name_string="".join(table_name)
    table_py_check=globals()[table_name_string]


    list_merged=setup+['Cut']+['Cut Level']+['Variable']
    table_py_check=table_py_check.sort_values(by = list_merged, na_position = 'first')
    table_xls_check=table_xls_check.sort_values(by = list_merged, na_position = 'first')   

    table_py_check=table_py_check.reset_index()
    table_xls_check=table_xls_check.reset_index()

In [17]:
#Cheking program when tables have same len

def check_func3_checks():
    
    #global table_check_merged

    #####CHECK 1 - WHEN SAME NUMBER OF ROWS

    if len(table_py_check)==len(table_xls_check):
        print("Checks for table ", table_name)
        print("same number of rows")
        print("all columns checekd and fine except for:")
        for v in table_xls_check.columns:
            try:
                check_equals=round(table_py_check[v],5).equals(round(table_xls_check[v],5))
                #print (v,"column is ok")
                if check_equals =="False":
                    print("Error 1 Float: ", v, "- variable has different data, please check")

            except TypeError: #check object variables here maybe we should control with dtype=object
                check_equals=table_py_check[v].equals(table_xls_check[v])
                if check_equals ==False:
                    print("Error 1 - object: ", v, "variable has different data, please check" )

            except KeyError:
                print("Error 2: ", v, "- variable is not checked, failed (column seems to be missed python table)")

    else: #program when we don't have same len or same rows
        print("Tables", table_name, "have different len - data not checked on total rows - check on shared rows is launched instead")
        
        
        
        
        #####CHECK 2 when not same len, DIFFERENT ROWS
        
        #merge in inner join
        
        list_merged=setup+['Cut']+['Cut Level']+['Variable']      #this variable already exists in previous function - make it global?

        table_check_merged=table_py_check.merge(table_xls_check,how='inner',on=list_merged)
        if len(table_check_merged)==0:
            print("Merge of files is failing for combination of ",list_merged) 
            print("Check you are using the same version of spss file that the one used for output. Specially Variable column with battery name")
        else:
            
            not_in_py=[]
            #table_check_merged=table_check_merged.reset_index()
            table_check_merged=round(table_check_merged,6)
            table_check_merged=table_check_merged.fillna(0)
            list_merged=setup+['Cut']+['Cut Level']+['Variable']


            print(table_name[0],"- checks:")

            for i in table_xls_check.columns:


                try:
                    if table_check_merged[i+"_x"].equals(table_check_merged[i+"_y"]):
                        continue
                    else: 

                        if i=="index":
                            continue
                        else:
                            print("Column - ",i," is failing on check")

                except KeyError:

                    if i in list_merged:
                        continue
                    else:
                        #continue
                        not_in_py.extend([i])


            if not_in_py==[]:
                print("Every columns in excel was checked and it is fine for rows where merge was possible")
                print("number of rows checked",len(table_check_merged)," out of ",len(table_xls_check),"in excel output and ",len(table_py_check)," in python output")
            else:        
                print("number of rows checked",len(table_check_merged)," out of ",len(table_xls_check),"in excel output and ",len(table_py_check)," in python output")
                print("The following columns in excel were not checked due to not existence in python table")
                print(not_in_py)


### PATTERNLIST - DEFINE LISTS

In [18]:
#PATTERNLIST DEFINITION


df["Weight_Final2"]=df["Weight_Final"]

Q320=list(df.filter(regex=r'^Q320_[0-9]$').columns)
Q600=list(df.filter(regex=r'^Q600_').columns)

#SCORES
Q320_sc=list(df.filter(regex=r'^Q320_\d+r_G_reb$').columns)
Q215_sc=list(df.filter(regex=r'^Q215_\d+r_G_reb$').columns)
Q410_sc=list(df.filter(regex=r'^Q410_\d+r_G_reb$').columns)


###list to remove
#####Q320_sc.remove('Q320_3r_G_reb')

list_sc=Q320_sc + Q215_sc
print(list_sc)

['Q320_1r_G_reb', 'Q320_2r_G_reb', 'Q320_3r_G_reb', 'Q320_4r_G_reb', 'Q320_5r_G_reb', 'Q320_6r_G_reb', 'Q320_7r_G_reb', 'Q320_8r_G_reb', 'Q320_9r_G_reb', 'Q320_10r_G_reb', 'Q320_11r_G_reb', 'Q320_12r_G_reb', 'Q320_13r_G_reb', 'Q320_14r_G_reb', 'Q320_15r_G_reb', 'Q320_16r_G_reb', 'Q320_17r_G_reb', 'Q320_18r_G_reb', 'Q320_19r_G_reb', 'Q320_20r_G_reb', 'Q320_21r_G_reb', 'Q320_22r_G_reb', 'Q320_23r_G_reb', 'Q215_2r_G_reb', 'Q215_3r_G_reb', 'Q215_4r_G_reb', 'Q215_5r_G_reb', 'Q215_6r_G_reb', 'Q215_7r_G_reb', 'Q215_8r_G_reb', 'Q215_9r_G_reb', 'Q215_10r_G_reb', 'Q215_11r_G_reb', 'Q215_15r_G_reb']


### TABLES PROGRAM - DEFINE THE TABLES YOU NEED:


In [19]:
###CALLING FUNCTION TO GENERATE TABLES

var=list_sc                                 #DEFINE LIST OF SCORES OR VARS
setup=["COUNTRY", "Company"]                #DEFINE HEADER
period=["Month"]
export="N"      # add to function           #"Y" If you want to export
Check_py_xls="N"                            #"Y" IF YOU WANT TO CHECK TABLES AGAINST EXCEL


###task create spaces in "table name" and make trim when constructing the variable


#cuts=["Gender","Age","Region"]              #DEFINE CUTS
cuts=["Q600A","Age","Region"]
table_name=['Scores']  ###name of table in python
tables_spss=['Scores'] ###name of table in excel - ONLY WHEN YOU WANT TO CHECK THE TABLE AGAINST SPSS EXCEL TABLE
Check_py_xls="N"
table_function()

########

# cuts=["Gender"]

# table_name=['ScoresGender']                 #for the moment not spaces allowed
# tables_spss=['Scores by Gender']            #excel table name
# Check_py_xls="N"
# table_function()

########

# cuts=["Overall","Q600_0","Region"]

# table_name=['ScoresClient']                 #for the moment not spaces allowed
# tables_spss=['Scores by clietn (some scores)'] 
# Check_py_xls="Y"
# table_function()



In [19]:
#####EXPORTATION OUTPUT PATH DEFINITION

#If export="Y":   ######include sentence to export or not export the tables
#C:\Users\JavierCabrera\Desktop\Prueba
export="Y"
if export=="Y":
    
    #reporting_path=output_path
    Output_name='/ Output From Python.xlsx'
    Excel_Output = pd.ExcelWriter(output_path + Output_name)

#Exportation

    #Scores.to_excel(Excel_Output,sheet_name='Scores Overall',index=False,float_format="%.6f")
    #ScoresGender.to_excel(Excel_Output,sheet_name='Scores Gender',index=False,float_format="%.6f")
    ScoresClient.to_excel(Excel_Output,sheet_name='Scores Client',index=False,float_format="%.6f")
    table_xls_check.to_excel(Excel_Output,sheet_name='Scores Client XLS',index=False)
    
    Excel_Output.save()

else:
    pass

NameError: name 'ScoresClient' is not defined

### CHEKING PROGRAM WITH EXCEL


In [ ]:
cut_check_xls=table_xls_check["Cut"].unique()
cut_check_py=table_py_check["Cut"].unique()
Variable_check_xls=table_xls_check["Variable"].unique()
Variable_check_py=table_py_check["Variable"].unique()

counter=0

print("advanced checks on Variable and Cut- if not mismacht please check file in spss is the same for output and python")

for i in cut_check_xls:
    
    if i in cut_check_py:
        continue
    else:
        if counter==0:
            printmd("**These cuts are not included in python and they are in xls:**")
            print(i)
        else:
            print(i)
    counter=counter+1

counter=0 

for i in cut_check_py:
    
    if i in cut_check_xls:
        continue
    else:
        if counter==0:
            printmd("**These cuts are not included in xls and they are in python:**")
            print(i)
        else:
            print(i)
    counter=counter+1

if counter==0:
    print("everthing fine for variable Cut")

    
###check variable


counter=0

for i in Variable_check_xls:
    
    if i in Variable_check_py:
        continue
    else:
        if counter==0:
            printmd("**These Variables are not included in python and they are in xls:**")
            print ("These Variables are not included in python and they are in xls:")
            print(i)
        else:
            print(i)
    counter=counter+1

counter=0 

for i in Variable_check_py:
    
    if i in Variable_check_xls:
        continue
    else:
        if counter==0:
            printmd("**These Variables are not included in xls and they are in python:**") 
            print(i)
        else:
            print(i)
    counter=counter+1

if counter==0:
    print("everthing fine for variable Variable")

##### DISTRIBUTIONS FUNCTIONS

In [21]:
def dist_tables(variable):
    global df_col_count
    global setup_cut
    global setup_cut_save
    
    setup_cut_save=setup_cut.copy()
    print (setup_cut_save)
    setup_cut.append(variable)
    print (setup_cut)
    
    df_col = pd.pivot_table(df, index=setup_cut,values=['Weight_Final'],columns=['Month'], aggfunc='count')
    df_col_count = pd.pivot_table(df, index=setup_cut,values=['Weight_Final'],columns=['Month'], aggfunc='count')   
    
    setup_cut=setup_cut_save.copy()
    
    print (setup_cut_save)
    print (setup_cut)
    
    ###Calculate percentage
    ###task maybe we shoulnd't multiply by 100.
    
    for i in df_col.columns:
        df_col[i]=(df_col[i]/df_col[i].groupby(level=1).transform(sum)*100).round(1)


    ###removing index 

    df_col.columns=df_col.columns.droplevel(0)
    df_col.columns.name=None
    df_col=df_col.reset_index()


    df_col_count.columns=df_col_count.columns.droplevel(0)
    df_col_count.columns.name=None
    df_col_count=df_col_count.reset_index()


    ###APPLY DICT FOR WAVES

    df_col.rename(columns=waves_dict,inplace=True)
    df_col_count.rename(columns=waves_dict,inplace=True)

    ###APPLY DICT FOR VALUES

    #dist
    df_col_2=df_col
    table_var=list(df_col.columns.values)
    counter=0

    for i in table_var:

        counter=counter+1

        try:
            df_col=df_col.replace({i:Val_lab[i]})

        except:
            pass

    #count
    df_col_count_2=df_col_count
    table_var=list(df_col_count.columns.values)
    counter=0

    for i in table_var:

        counter=counter+1

        try:
            df_col_count=df_col_count.replace({i:Val_lab[i]})

        except:
            pass        


    #######CREATE VAR NAME and level variable
    #task automate this part

    ###alert if "VARIABLE" exist will be removed
    if 'VARIABLE' in df_col.columns:
        df_col.drop(columns=['VARIABLE'],inplace=True) ###removing vars

    ###insert columns
    df_col.insert(len(setup)+1,"VARIABLE",variable)
    df_col.rename(columns={variable:"Level"},inplace=True)

    #count
    ###alert if "VARIABLE" exist will be removed
    if 'VARIABLE' in df_col_count.columns:
        df_col_count.drop(columns=['VARIABLE'],inplace=True) ###removing vars

    ###insert columns
    df_col_count.insert(2,"VARIABLE",variable)
    df_col_count.rename(columns={variable:"Level"},inplace=True)



    ### APPLY DICT FOR VARIABLES        
    df_col=df_col.replace({"VARIABLE":dict_var})
    df_col_count=df_col_count.replace({"VARIABLE":dict_var})      


    #### Columns names as text

    df_col.columns
    shape_df_col=df_col.shape
    ncolumns=shape_df_col[1]
    ###variables=3

    df_col_count.columns
    shape_df_col_count=df_col_count.shape
    ncolumns_c=shape_df_col_count[1]


    for i in range(ncolumns):

        name=str(df_col.columns[i])
        df_col.rename(columns={ df_col.columns[i]: name }, inplace = True)
        #print(name)

    for i in range(ncolumns_c):

        name=str(df_col_count.columns[i])
        df_col_count.rename(columns={ df_col_count.columns[i]: name }, inplace = True)
        #print(name)

    # changing Nan to 0s
    df_col=df_col.fillna(0)
    
    return df_col

In [ ]:
#####27 April - Mirar esta parte que creemos que es la que duplica el level
#### Ver si funciona para otros cortes
####ver los counts como integrarlos,

    #######CREATE VAR NAME and level variable
    #task automate this part

    ###alert if "VARIABLE" exist will be removed
    if 'VARIABLE' in df_col.columns:
        df_col.drop(columns=['VARIABLE'],inplace=True) ###removing vars

    ###insert columns
    df_col.insert(len(setup)+1,"VARIABLE",variable)
    df_col.rename(columns={variable:"Level"},inplace=True)

    #count
    ###alert if "VARIABLE" exist will be removed
    if 'VARIABLE' in df_col_count.columns:
        df_col_count.drop(columns=['VARIABLE'],inplace=True) ###removing vars

    ###insert columns
    df_col_count.insert(2,"VARIABLE",variable)
    df_col_count.rename(columns={variable:"Level"},inplace=True)



In [52]:
####TEST TABLES ONLY WITH FIRST FUCNTION

setup=["COUNTRY", "Company"] 
var=['Q600_0','Q600_24']

cuts=['Gender']
df_col=dist_tables('Q600_0')

#df_col=col_join(var)
#df_col=col_join_cut(var,cuts)
df_col.tail(2)

['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender', 'Q600_0']
['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender']


,COUNTRY,Company,Gender,VARIABLE,Level,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021
14,Mexico,Company 4,Female,"In the last 12 months, have you used or purcha...",No,4.8,3.6,3.0,2.4,1.2,3.0,5.4,3.0,1.2,4.8,3.0,4.8
15,Mexico,Company 4,Female,"In the last 12 months, have you used or purcha...",Yes,52.7,47.9,55.7,55.1,49.7,53.9,50.9,52.7,46.7,50.3,56.6,50.3


In [22]:
def col_join(var):
    ###concat for each var
    
    counter_var=0
    
    for i in var:
        
        df_col=dist_tables(i)
    
        if counter_var==0:
        
            df_col2=df_col
        
        else:
           
            df_col2=pd.concat([df_col2,df_col],ignore_index=False)
        
        counter_var=counter_var + 1

    return df_col2

def col_join_cut(var,cuts):
    #iteration for cuts
    global setup_cut
    
    counter_cut=0
    for i in cuts:
        setup_cut=setup.copy()
        setup_cut.append(i)
        df_col=col_join(var)
        
        if counter_cut==0:
            
            df_col2=df_col
            
        else:
            df_col2=pd.concat([df_col2,df_col],ignore_index=False)
        counter_cut=counter_cut + 1
        
    return df_col
        

In [ ]:
23 MARZO ----- intentando incluir el corte cuts en la función

    - ver si podemos renombrar Gender como cut en la iteración 


los counts tendremos que añadirlos
luego los 0s.

In [44]:
df['Q600_0'].describe()
df['Q600_0'].values
df['Q600_0'].unique()

array([1., 0.])

In [51]:
df_col_test = pd.pivot_table(df, index=["COUNTRY", "Company","Gender",'Q600_0'],values=['Weight_Final'],columns=['Month'], aggfunc='count')
df_col_test

Weight_Final                                \
Month                                 67.0  68.0  69.0  70.0  71.0  72.0   
COUNTRY Company Gender Q600_0                                              
30.0    551.0   1.0    0.0             1.0   2.0   2.0   NaN   2.0   1.0   
                       1.0            46.0  51.0  47.0  48.0  42.0  41.0   
                2.0    0.0             NaN   1.0   2.0   3.0   NaN   1.0   
                       1.0            53.0  46.0  49.0  49.0  56.0  57.0   
        832.0   1.0    0.0             1.0   2.0   1.0   2.0   2.0   5.0   
                       1.0            46.0  33.0  45.0  41.0  40.0  45.0   
                2.0    0.0             1.0   2.0   3.0   2.0   3.0   2.0   
                       1.0            52.0  63.0  51.0  56.0  55.0  48.0   
        911.0   1.0    0.0             4.0  10.0   5.0   3.0   5.0   NaN   
                       1.0            44.0  38.0  48.0  41.0  44.0  46.0   
                2.0    0.0             8.0  11.0   7.0   8.0   5.0   4.0   
                       1.0            45.0  41.0  41.0  48.0  46.0  51.0   
        1984.0  1.0    0.0             3.0   6.0   2.0  10.0   3.0   5.0   
                       1.0            68.0  75.0  67.0  61.0  79.0  67.0   
                2.0    0.0             8.0   6.0   5.0   4.0   2.0   5.0   
                       1.0            88.0  80.0  93.0  92.0  83.0  90.0   

                                                                   
Month                          73.0  74.0  75.0  76.0  77.0  78.0  
COUNTRY Company Gender Q600_0                                      
30.0    551.0   1.0    0.0      1.0   3.0   1.0   1.0   NaN   NaN  
                       1.0     88.0  65.0  69.0  66.0  76.0  87.0  
                2.0    0.0      3.0   1.0   2.0   1.0   NaN   2.0  
                       1.0     61.0  85.0  83.0  77.0  78.0  74.0  
        832.0   1.0    0.0      4.0   4.0   5.0   4.0   1.0   5.0  
                       1.0     71.0  63.0  63.0  65.0  73.0  79.0  
                2.0    0.0      5.0   4.0   4.0   3.0   7.0   4.0  
                       1.0     67.0  83.0  84.0  72.0  69.0  74.0  
        911.0   1.0    0.0      4.0   5.0   3.0   8.0   5.0   2.0  
                       1.0     49.0  42.0  44.0  33.0  36.0  44.0  
                2.0    0.0     11.0  11.0   7.0  10.0  11.0   8.0  
                       1.0     36.0  42.0  46.0  50.0  48.0  46.0  
        1984.0  1.0    0.0      3.0   3.0   9.0   5.0   5.0   5.0  
                       1.0     70.0  72.0  78.0  70.0  62.0  70.0  
                2.0    0.0      9.0   5.0   2.0   8.0   5.0   8.0  
                       1.0     85.0  89.0  78.0  84.0  94.0  84.0

In [34]:
setup=["COUNTRY", "Company"] 
var=['Q600_0','Q600_24']

cuts=['Gender']
#df_col=dist_tables('Q600_0')

#df_col=col_join(var)
df_col=col_join_cut(var,cuts)
df_col.tail(2)

['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender', 'Q600_0']
['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender', 'Q600_24']
['COUNTRY', 'Company', 'Gender']
['COUNTRY', 'Company', 'Gender']


,COUNTRY,Company,Gender,VARIABLE,Level,July 2020,August 2020,September 2020,October 2020,November 2020,December 2020,January 2021,February 2021,March 2021,April 2021,May 2021,June 2021
14,Mexico,Company 4,Female,Company annual report - Company annual report,No,56.3,50.9,58.1,56.3,49.7,56.3,56.3,53.8,47.3,55.1,57.8,55.1
15,Mexico,Company 4,Female,Company annual report - Company annual report,Yes,1.2,0.6,0.6,1.2,1.2,0.6,0.0,1.8,0.6,0.0,1.8,0.0


In [53]:
#####EXPORTATION OUTPUT PATH DEFINITION

export="Y"

#If export="Y":   ######include sentence to export or not export the tables
#C:\Users\JavierCabrera\Desktop\Prueba
export="Y"
if export=="Y":
    
    #reporting_path=output_path
    Output_name='/ Output From Python.xlsx'
    Excel_Output = pd.ExcelWriter(output_path + Output_name)

#Exportation

    #Scores.to_excel(Excel_Output,sheet_name='Scores Overall',index=False,float_format="%.6f")
    #ScoresGender.to_excel(Excel_Output,sheet_name='Scores Gender',index=False,float_format="%.6f")
    df_col.to_excel(Excel_Output,sheet_name='Scores Client',index=False,float_format="%.6f")
    
    Excel_Output.save()

else:
    pass

In [34]:

table_py_check.head()

,index,COUNTRY,Company,Cut,Cut Level,Variable,Mean_July 2020,Valid N_July 2020,Mean_August 2020,Valid N_August 2020,...,Mean_June 2021,Valid N_June 2021,ValidN_L3M,Mean_L3M,ValidN_L6M,Mean_L6M,ValidN_L9M,Mean_L9M,ValidN_L12M,Mean_L12M
0,208,Mexico,Company 1,Age,18-24,Acts responsibly to protect the environment,74.027445,26.106379,64.715478,28.061851,...,63.016540,29.296366,89.558434,66.550778,202.890178,67.454133,285.592339,65.396032,362.840704,66.300774
1,96,Mexico,Company 1,Age,18-24,Adapts quickly to change,76.514339,28.147506,71.716196,29.174230,...,76.754008,28.396966,85.383482,77.570285,209.956686,78.856395,302.923249,76.596603,384.439872,76.022354
2,176,Mexico,Company 1,Age,18-24,Behaves ethically,78.087691,26.128730,72.970981,30.270837,...,73.029180,31.183601,93.589141,74.972744,217.193795,74.814999,309.116270,73.004680,388.595971,73.511558
3,400,Mexico,Company 1,Age,18-24,Benefit Of Doubt,76.393649,29.223946,68.646463,31.398989,...,70.671361,30.195765,95.727470,75.790968,221.124927,74.361591,312.139096,71.694668,399.162876,71.828105
4,432,Mexico,Company 1,Age,18-24,Buy,83.243069,31.309774,78.435229,33.639520,...,79.673592,31.095165,98.744733,83.858748,229.155962,84.866846,324.883493,81.735904,416.233633,81.513566


In [ ]:
cut_check_xls=table_xls_check["Cut"].unique()
cut_check_py=table_py_check["Cut"].unique()

In [ ]:
#crear programa para chequear cada columna - y luego pasar listado de variables con fallo.
#REPORTAR COLUMNAS QUE ESTÁN EN UNA LISTA Y NO ESTÁN EN LA OTRA
#REPORTAR COLUMNAS CON FALLO EN EL CHEQUEO.



In [48]:
Q320 - 

,COUNTRY,Company,Cut,Cut Level,Variable,Mean_July 2020,Valid N_July 2020,Mean_August 2020,Valid N_August 2020,Mean_September 2020,...,Mean_June 2021,Valid N_June 2021,ValidN_L3M,Mean_L3M,ValidN_L6M,Mean_L6M,ValidN_L9M,Mean_L9M,ValidN_L12M,Mean_L12M
0,Mexico,Company 1,Gender,Male,Offers high quality products and services,74.423672,47.827765,74.025564,54.540216,74.871793,...,80.744225,80.639743,218.452156,79.028814,434.795498,77.514202,570.843790,74.911042,721.597367,74.793367
1,Mexico,Company 1,Gender,Female,Offers high quality products and services,80.302730,52.578493,76.279987,44.399489,74.353366,...,77.570038,76.627599,228.363955,79.605206,468.901134,80.132055,636.356045,78.308770,784.464502,77.976251
2,Mexico,Company 2,Gender,Male,Offers high quality products and services,76.943587,47.789787,82.197045,35.788651,78.691112,...,79.599395,79.722663,217.947661,79.413915,412.339350,77.887381,546.777078,77.481608,675.811170,77.930518
3,Mexico,Company 2,Gender,Female,Offers high quality products and services,82.125285,50.677458,78.960689,65.117380,78.861290,...,80.929276,79.626158,227.123540,80.837128,480.923170,80.816209,648.596187,80.067983,817.456321,80.046593
4,Mexico,Company 3,Gender,Male,Offers high quality products and services,74.728476,46.642951,73.662737,47.211082,71.266684,...,73.702965,43.412039,127.817301,65.959064,274.943860,68.933458,411.930674,68.397609,557.429891,69.603032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,Mexico,Company 4,Region,Mexico México,In an discussion or argument I would defend,62.337130,26.917322,62.491652,21.935426,71.364895,...,64.342590,28.774927,85.056833,65.844524,174.214046,65.856233,249.630034,66.282617,320.394775,66.061436
812,Mexico,Company 4,Region,Mexico Jalisco,In an discussion or argument I would defend,68.675816,13.001677,55.705533,8.809977,55.098295,...,68.368760,16.239147,45.153638,56.115629,85.671895,57.357112,124.214791,59.886981,159.223095,59.871873
813,Mexico,Company 4,Region,Mexico Veracruz,In an discussion or argument I would defend,80.478968,6.068331,29.395229,5.875027,76.206222,...,36.037620,7.499292,21.925840,60.404940,48.755625,57.365307,72.043976,59.537081,88.997994,60.159512
814,Mexico,Company 4,Region,Mexico Puebla,In an discussion or argument I would defend,73.762352,6.152616,70.642044,15.120450,61.918596,...,65.034190,9.593112,27.284195,69.371348,60.007564,61.544592,89.692079,60.588411,116.207265,62.634891


In [ ]:
###FUNCIÓN ENCONTRAR BARRA BAJA...

Q320_4
Q320_3_24
Q320_5_54

In [ ]:
#tasks to work out
# allow spaces in table name - done - integrar
# USE IN CUTS THE DICTIONARY LABELS - done
# SEE HOW TO REMOVE VARS FROM LISTS - function? - done integrar
# Add patternlist to variable name or wether create column with code and remove patternlist name - done integrar

# improve code, architecture - pycharm
# add distributions

# CONTINTUE WORKING IN MERGE PROGRAM - include it in function
# CREATE AND ADVANCE CHECK FUNCTION
# create battery column in python table
# checks - convert into functions    - done
# export in function to get table name
# test automated excel report and create function for it. 
# create another type of check when number of rows are different    - DONE


####done
# Work out program with several tables defined by user - done 
#split head between  set-up and cuts with loop over the different cuts - done
        #we could keep name of head in loop of set up + cuts
        #for checking we would need to create another range variable for cuts with a sort of look up function that give each cut a number sort by cut, we cut sort within the loop of cuts as well by nowdays procedure
        #control excel creation columns
#when sorting there is a manual selection of column - done
#when sorting there is a manual definition of where the column "variable" must be created - done
#round when exporting (not for datacheking) - truncated when exporting but not with all decimals - done



########ideas!!
###include in function possibility of launch tables with periods in row
###meta data read variable type
###include weithg final = 1 when exception
###MISSING VARAIBLES

In [ ]:
##import path

####table_final2.to_excel('C:/Users/JavierCabrera/Desktop/Prueba/Scores Weighted.xlsx',sheet_name='Scores weighted')

In [ ]:
excel_table_formatting()

In [ ]:
#two types of checks - first if same number of rows - sorting 
#if different numbers of rows - check mergin on columns

In [ ]:
check_func3_checks()

In [ ]:
###export tables
#ScoresGender["Cut"].to_excel('C:/Users/JavierCabrera/Desktop/Notebooks/Pivot tables project/check folder/merged check table3.xlsx',sheet_name='py')
#table_xls_check["Cut"].to_excel('C:/Users/JavierCabrera/Desktop/Notebooks/Pivot tables project/check folder/merged check table4.xlsx',sheet_name='xls')

In [ ]:
####table_xls_check["Battery"].drop

In [ ]:
#####COLUMN % CALCULATION####